## Task3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from UsefulFunctions import LoadData, PricesDK

### 3.1

In [ ]:
# Load the necessary data
df_prices, df_pro = LoadData()

# Compute Buy Prices using the given function
df_prices = PricesDK(df_prices)

# Keep only necessary columns
df_prices = df_prices[["HourDK", "Buy" , "Sell"]] # Keep only the Buy and Sell columns
df_pro = df_pro[["HourDK", "Load", "PV"]] # Consumption culumn is renamed to Load in the PricesDK function

# Merge both datasets on time
df = pd.merge(df_pro, df_prices, on="HourDK", how="inner")
del df_pro, df_prices # Remove now unused dataframes

# Convert time column to datetime format
df["HourDK"] = pd.to_datetime(df["HourDK"])

# Filter data for the years 2022 and 2023
# df_filtered = df[(df["HourDK"].dt.year == 2022) | (df["HourDK"].dt.year == 2023)]
# this is already done in the PricesDK function

# Compute total cost for each year (Consumption * Buy Price per hour)
df["TotalCost"] = df["Load"] * df["Buy"]
annual_total_costs = df.groupby(df["HourDK"].dt.year)["TotalCost"].sum()

# Compute total yearly consumption and average Buy Price
annual_consumption = df.groupby(df["HourDK"].dt.year)["Load"].sum()
annual_avg_buy_price = df.groupby(df["HourDK"].dt.year)["Buy"].mean()

# Compute estimated cost using total consumption * average Buy Price
rough_costs = annual_consumption * annual_avg_buy_price

# Analyze the difference
difference = abs(annual_total_costs - rough_costs)

# Compare the two methods
comparison_df = pd.DataFrame({
    "Exact Total Cost": annual_total_costs,
    "Total Consumption (kWh)": annual_consumption,
    "Average Buy Price (DKK/kWh)": annual_avg_buy_price,
    "Estimated Total Cost (Rough Calculation)": rough_costs,
    "Difference": difference
})

# Print the final comparison table
print(comparison_df)

### 3.2

In [ ]:
# Compute the net load
df["Net"] = df["Load"] - df["PV"]

# Compute the total cost for each hour
df.loc[(df["Net"] > 0), "TotalCost-3.2"] = df["Net"] * df["Buy"] # Buy
df.loc[(df["Net"] < 0), "TotalCost-3.2"] = df["Net"] * df["Sell"] # Sell

# Compute the total cost for each year
annual_total_costs_PV = df.groupby(df["HourDK"].dt.year)["TotalCost-3.2"].sum()

# Compute the benefit of having PV panels
PV_benefit = annual_total_costs - annual_total_costs_PV

# Display the results
display(annual_total_costs_PV)
PV_benefit

### 3.3

In [ ]:
def Optimizer(params, ps, pb, net_load):

    """ 
    Calculate the dimension of your decision variables (n)
    # Do not hard-code values (i.e. n = 24!)
    # A day may have 23 or 25 hours or you may want to solve your problem over 48 hours!
    """
    
    n = len(ps)
    
    ### Define the decision variables ###
    p_c = cp.Variable(n)
    p_d = cp.Variable(n)
    X   = cp.Variable(n)
    
    pos_net_load = np.maximum(0, net_load)
    neg_net_load = -np.maximum(0, -net_load)

    ### Define the cost function ###
    profit = cp.sum((p_d + neg_net_load)@ps - (p_c + pos_net_load)@pb)
    
    ### Add constraints ### 
    constraints = [p_c >= 0, 
                   p_d >= 0, 
                   p_c <= params['Pmax'], 
                   p_d <= params['Pmax']]
    constraints += [X >= 0, X <= params['Cmax']]
    constraints += [X[0]==params['C_0'] + p_c[0]*params['n_c'] - p_d[0]/params['n_d']]
    constraints += [X[1:] == X[:-1] + p_c[1:]*params['n_c'] - p_d[1:]/params['n_d']]
    constraints += [X[n-1]>=params['C_n']]
    
    ### Solve the problem ###
    problem = cp.Problem(cp.Maximize(profit), constraints)
    problem.solve(solver=cp.CLARABEL)
    
    return profit.value, p_c.value, p_d.value, X.value

In [ ]:
# Load data
df_prices, df_pro = LoadData()
df_prices = PricesDK(df_prices)

# Battery parameters
battery_params = {
    'Pmax': 5,      # Power capacity in kW
    'Cmax': 10,     # Energy capacity in kWh
    'C_0': 5,       # Initial SOC (50%)
    'C_n': 5,       # Final SOC (50%)
    'n_c': 0.95,    # Charging efficiency
    'n_d': 0.95     # Discharging efficiency
}

# Initialize results dictionaries
costs_pv_only = {'2022': 0, '2023': 0}
costs_pv_battery = {'2022': 0, '2023': 0}

# Process data by days
days = pd.to_datetime(df_prices['HourDK'].dt.date.unique())

for day in days:
    year = day.year
    day_date = day.date()
    
    # Filter data for the current day
    prices_day = df_prices[df_prices['HourDK'].dt.date == day_date]
    pro_day = df_pro[df_pro['HourDK'].dt.date == day_date]
    
    # Make sure we have both price and prosumer data for this day
    if len(prices_day) == 0 or len(pro_day) == 0:
        continue
        
    # Make sure the timestamps match
    merged = pd.merge(prices_day, pro_day, on='HourDK')
    if len(merged) == 0:
        continue
    
    # Extract data
    buy_prices = merged['Buy'].values
    sell_prices = merged['Sell'].values
    load = merged['Load'].values
    generation = merged['PV'].values
    
    # Calculate net load (positive means consumption exceeds generation)
    net_load = load - generation
    
    # Calculate cost with PV only (under net metering)
    day_cost_pv_only = 0
    for i in range(len(net_load)):
        if net_load[i] > 0:  # Buying from grid
            day_cost_pv_only += net_load[i] * buy_prices[i]
        else:  # Selling to grid
            day_cost_pv_only += net_load[i] * sell_prices[i]  # Net_load is negative here
    
    # Optimize battery operation with price arbitrage
    # For net metering, the goal is to charge when prices are low and discharge when prices are high
    profit_value, p_c_value, p_d_value, X_value = Optimizer(battery_params, buy_prices)
    
    # Calculate new net load with battery
    net_load_with_battery = net_load + p_c_value - p_d_value
    
    # Calculate cost with PV and battery
    day_cost_pv_battery = 0
    for i in range(len(net_load_with_battery)):
        if net_load_with_battery[i] > 0:  # Buying from grid
            day_cost_pv_battery += net_load_with_battery[i] * buy_prices[i]
        else:  # Selling to grid
            day_cost_pv_battery += net_load_with_battery[i] * sell_prices[i]
    
    # Add to yearly totals
    costs_pv_only[str(year)] += day_cost_pv_only
    costs_pv_battery[str(year)] += day_cost_pv_battery

# Calculate yearly benefits of having a battery
benefits = {
    '2022': costs_pv_only['2022'] - costs_pv_battery['2022'],
    '2023': costs_pv_only['2023'] - costs_pv_battery['2023']
}

# Display results
print(f"Annual costs with PV only (DKK):")
print(f"2022: {costs_pv_only['2022']:.2f}")
print(f"2023: {costs_pv_only['2023']:.2f}")
print("\nAnnual costs with PV and battery (DKK):")
print(f"2022: {costs_pv_battery['2022']:.2f}")
print(f"2023: {costs_pv_battery['2023']:.2f}")
print("\nAnnual benefits from battery (DKK):")
print(f"2022: {benefits['2022']:.2f}")
print(f"2023: {benefits['2023']:.2f}")

# Visualize the comparison
years = ['2022', '2023']
pv_costs = [costs_pv_only['2022'], costs_pv_only['2023']]
pv_bat_costs = [costs_pv_battery['2022'], costs_pv_battery['2023']]
battery_benefits = [benefits['2022'], benefits['2023']]

plt.figure(figsize=(10, 6))
x = np.arange(len(years))
width = 0.35

plt.bar(x - width/2, pv_costs, width, label='PV Only')
plt.bar(x + width/2, pv_bat_costs, width, label='PV + Battery')

plt.xlabel('Year')
plt.ylabel('Annual Cost (DKK)')
plt.title('Annual Costs: PV Only vs. PV + Battery')
plt.xticks(x, years)
plt.legend()

plt.figure(figsize=(10, 6))
plt.bar(years, battery_benefits)
plt.xlabel('Year')
plt.ylabel('Annual Benefit (DKK)')
plt.title('Annual Benefit from Adding Battery')

plt.tight_layout()
plt.show()